In [18]:
import numpy as np
import matplotlib.pyplot as plt

In [19]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

In [20]:
x1 = np.random.randn(200)
x2 = np.random.randn(200)
x3 = np.random.randn(200)
x4 = np.random.randn(200)
y = np.zeros_like(x1)

for i in range(0, len(x1)):
    y[i] = 3*x1[i] + 2*x2[i] + 4*x3[i] + 7*x4[i]


In [21]:
data = [x1, x2, x3, x4, y]
data = np.transpose(data)

In [22]:
train_input = data[:150, :4]
train_target = data[:150, 4]

In [23]:
test_input = data[150:, :4]
test_target = data[150:, 4]

In [24]:
torch.manual_seed(1234)

In [25]:
# N = 1 # Batch size
D_in = 4 # Input dimension
D_out = 1 # Output dimension 
hidden_units = 3

In [35]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 12)
        self.fc2 = nn.Linear(12, 6)
        self.fc3 = nn.Linear(6, 2)
        self.fc4 = nn.Linear(2, 1)
        
    def forward(self, x):
        x = self.fc1(x).clamp(min = 0)
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = self.fc4(x)
        return(x)

net = Net()

In [36]:
# Choose optmizer and loss function
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr = 0.1, momentum = 0.9) # lr = learning rate 

In [37]:
# Train
epochs = 50

for epoch in range(epochs):
    inputs = torch.autograd.Variable(torch.Tensor(train_input).float())
    targets = torch.autograd.Variable(torch.Tensor(train_target).long())
    
    optimizer.zero_grad()
    out = net(inputs)
    loss = criterion(out, targets)
    loss.backward()
    optimizer.step()
    
    if epoch == 0 or (epoch + 1) % 10 == 0:
        print('Epoch %d Loss: %.4f' % (epoch + 1, loss.item()))

RuntimeError: Assertion `cur_target >= 0 && cur_target < n_classes' failed.  at ..\aten\src\THNN/generic/ClassNLLCriterion.c:92

In [38]:
# Avaliando a acurácia do modelo
inputs = torch.autograd.Variable(torch.Tensor(test_input).float())
targets = torch.autograd.Variable(torch.Tensor(test_target).long())

optimizer.zero_grad()
out = net(inputs)
_, predicted = torch.max(out.data, 1)

error_count = test_target.size - np.count_nonzero((targets == predicted) .numpy())
print('Errors: %d; Accuracy: %d%%' % (error_count, 100 * torch.sum(targets == predicted) / test_target.size))

Errors: 45; Accuracy: 10%
